# Dance Diffusion finetune

Licensed under the MIT License

Copyright (c) 2022 Zach Evans

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.


# Set Up

In [1]:
#@title Check GPU Status
import subprocess
simple_nvidia_smi_display = True#@param {type:"boolean"}
if simple_nvidia_smi_display:
    #!nvidia-smi
    nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output)
else:
    #!nvidia-smi -i 0 -e 0
    nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output)
    nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_ecc_note)

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-4ee632cc-361a-5a00-031b-cb051474e47d)



In [2]:
#@title Prepare folders
import subprocess, os, sys, ipykernel

def gitclone(url, targetdir=None):
    if targetdir:
        res = subprocess.run(['git', 'clone', url, targetdir], stdout=subprocess.PIPE).stdout.decode('utf-8')
    else:
        res = subprocess.run(['git', 'clone', url], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

def pipi(modulestr):
    res = subprocess.run(['pip', 'install', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

def pipie(modulestr):
    res = subprocess.run(['git', 'install', '-e', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

def wget(url, outputdir):
    res = subprocess.run(['wget', url, '-P', f'{outputdir}'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

try:
    from google.colab import drive
    print("Google Colab detected. Using Google Drive.")
    is_colab = True
    google_drive = True #@param {type:"boolean"}
    #@markdown Click here if you'd like to save the diffusion model checkpoint file to (and/or load from) your Google Drive:
    save_models_to_google_drive = True #@param {type:"boolean"}
except:
    is_colab = False
    google_drive = False
    save_models_to_google_drive = False
    print("Google Colab not detected.")

if is_colab:
    if google_drive is True:
        drive.mount('/content/drive')
        root_path = '/content/drive/MyDrive/AI/Bass_Diffusion'
    else:
        root_path = '/content'
else:
    root_path = os.getcwd()

import os
def createPath(filepath):
    os.makedirs(filepath, exist_ok=True)

initDirPath = f'{root_path}/init_audio'
createPath(initDirPath)
outDirPath = f'{root_path}/audio_out'
createPath(outDirPath)

if is_colab:
    if google_drive and not save_models_to_google_drive or not google_drive:
        model_path = '/content/models'
        createPath(model_path)
    if google_drive and save_models_to_google_drive:
        model_path = f'{root_path}/models'
        createPath(model_path)
else:
    model_path = f'{root_path}/models'
    createPath(model_path)

Google Colab detected. Using Google Drive.
Mounted at /content/drive


In [ ]:
#@title Install dependencies
!git clone https://github.com/harmonai-org/sample-generator
!pip install /content/sample-generator

# Train

In [ ]:
#@markdown Log in to [Weights & Biases](https://wandb.ai/) for run tracking
!wandb login

In [ ]:
%cd /content/sample-generator/
#@markdown Name for the finetune project
NAME="dd-drums-finetune"#@param {type:"string"}

#@markdown Path to the directory of audio data to use for fine-tuning
TRAINING_DIR="/content/drive/MyDrive/Audio/Drums"#@param {type:"string"}

#@markdown Path to the checkpoint to fine-tune
CKPT_PATH="/content/drive/MyDrive/AI/models/jmann-small-190k.ckpt"#@param {type:"string"}

#@markdown Number of training steps between demos
DEMO_EVERY=250 #@param {type:"number"}

#@markdown Number of training steps between saving model checkpoints
CHECKPOINT_EVERY=500 #@param {type:"number"}

#@markdown Batch size to fine-tune (make it as high as it can go for your GPU)
BATCH_SIZE=2 #@param {type:"number"}

#@markdown Accumulate gradients over n batches, useful for training on one GPU. 
#@markdown
#@markdown Effective batch size is BATCH_SIZE * ACCUM_BATCHES.
ACCUM_BATCHES=4 #@param {type:"number"}

!python3 /content/sample-generator/train_uncond.py --ckpt-path $CKPT_PATH --name $NAME --training-dir $TRAINING_DIR --sample-size 65536 --sample-rate 48000 --batch-size $BATCH_SIZE --demo-every $DEMO_EVERY --demo-steps 100 --checkpoint-every $CHECKPOINT_EVERY --num-workers 12 